In [2]:
import scanpy as sc
import pandas as pd
import scipy.io
import os

In [ ]:
data_dir = "/mnt/bench/filtered_matrices_mex/hg19"

# Step 1: load 10x matrix
adata = sc.read_10x_mtx(data_dir, var_names='gene_symbols', cache=True)

In [ ]:
# Step 2: load ground truth label
labels = pd.read_csv(os.path.join(data_dir, '68k_pbmc_barcodes_annotation.tsv'), sep='\t',usecols=[2, 3])
labels.columns = ['barcode', 'celltype']
labels = labels.set_index('barcode')

In [ ]:
# Step 3: assign label to AnnData
adata.obs['barcode'] = adata.obs_names
adata.obs = adata.obs.join(labels, on='barcode')
adata = adata[~adata.obs['celltype'].isna(), :]
adata.obs['celltype'] = adata.obs['celltype'].astype('category')

/tmp/ipykernel_15159/824238848.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['celltype'] = adata.obs['celltype'].astype('category')


In [ ]:
# Step 4: preprocessing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]
sc.tl.pca(adata, svd_solver='arpack')

# save to h5ad
adata.write("zheng68k_preprocessed.h5ad")

/root/miniconda3/envs/scvi-env/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:317: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm[key_obsm] = X_pca
